# Find well priced NFTs on rising collections

**Old calculation**
### ThorGuards #2716
- Date: 15.11.2021
- Price: 1.75 ETH
- URL: https://opensea.io/assets/0xa98b29a8f5a247802149c268ecf860b8308b7291/390

**Using Rarity tools calculation**
### Boss Beauties #2380
- Date: 15.11.2021
- Price: 4.9 ETH
- URL: https://opensea.io/assets/0xb5c747561a185a146f83cfff25bdfd2455b31ff4/2380

### Gutter Dog #1290
- Date: 15.11.2021
- Price: 1.65 ETH
- URL: https://opensea.io/assets/0x6e9da81ce622fb65abf6a8d8040e460ff2543add/1290

### DeadFellaz #8384
- Date: 15.11.2021
- Price: 1.29 ETH
- URL: https://opensea.io/assets/0x2acab3dea77832c09420663b0e1cb386031ba17b/8545

### Kong #7981
- Date: 15.11.2021
- Price: 69.42 ETH
- URL: https://opensea.io/assets/0xef0182dc0574cd5874494a120750fd222fdb909a/7981

### Cryptovoxels 1 Rack Pass
- Date: 15.11.2021
- Price: 3.5 ETH
- URL: https://opensea.io/assets/0x79986af15539de2db9a5086382daeda917a9cf0c/2089

### The Shiboshis
- Date: 15.11.2021
- Price:  ETH
- URL: 

In [1]:
import os
from datetime import datetime
from dateutil.parser import parse
import time

import numpy as np
import pandas as pd
from scipy import interpolate
from scipy.optimize import curve_fit
import scipy.stats as scs
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pprint import pprint
from tqdm import tqdm

import seaborn as sns
sns.set_theme(style="whitegrid")

from src.nft_analytics import NFTAnalytics
from src.infura_api import InfuraAPI

In [2]:
items_in_collection = 10000
nft = NFTAnalytics("0x11450058d796b02eb53e65374be59cff65d3fe7f")
asset_data = nft.fetch_data(max_offset=items_in_collection)

2021-12-19 11:47:18,353 - src.ethereum_api - INFO - Connected to Infura endpoint.


10000: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 201/201 [04:40<00:00,  1.39s/it]


In [3]:
asset_data2 = nft.fetch_data(max_offset=1)

0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.12s/it]


In [4]:
asset_data2

[{'id': 72383869,
  'token_id': '9999',
  'num_sales': 0,
  'background_color': None,
  'image_url': 'https://lh3.googleusercontent.com/yW36x8WP-Nk9AkPq1ukWFefXA0UQy1BUJsqzPJDjmS7KhI6GPOBjjNMOv3b_I7gZrSZsI9tYy1QBmcCUj7lhmyLLBFu7ZwUTU9Mn',
  'image_preview_url': 'https://lh3.googleusercontent.com/yW36x8WP-Nk9AkPq1ukWFefXA0UQy1BUJsqzPJDjmS7KhI6GPOBjjNMOv3b_I7gZrSZsI9tYy1QBmcCUj7lhmyLLBFu7ZwUTU9Mn=s250',
  'image_thumbnail_url': 'https://lh3.googleusercontent.com/yW36x8WP-Nk9AkPq1ukWFefXA0UQy1BUJsqzPJDjmS7KhI6GPOBjjNMOv3b_I7gZrSZsI9tYy1QBmcCUj7lhmyLLBFu7ZwUTU9Mn=s128',
  'image_original_url': 'https://shiboshis.mypinata.cloud/ipfs/QmWHfiwiEy7p3TT62dXSGFGwAKAmjwuy9v57wmtsbEVqwF',
  'animation_url': None,
  'animation_original_url': None,
  'name': '#9999',
  'description': 'SHIBOSHIS - Official Collection',
  'external_link': None,
  'asset_contract': {'address': '0x11450058d796b02eb53e65374be59cff65d3fe7f',
   'asset_contract_type': 'non-fungible',
   'created_date': '2021-10-14T21:13:37.

In [ ]:
df = nft.calculate_rarity_df(asset_data, items_in_collection)

In [ ]:
df = pd.DataFrame(columns=["Name", "Price", "Rarity", "RarityPriceRatio"])

for idx, asset in enumerate(asset_data):
    if asset["sell_orders"]:
        if asset["sell_orders"][0]["payment_token_contract"]["symbol"] == "ETH":
            price = float(asset["sell_orders"][0]["current_price"]) / 1e18
            if price != 0:
                rarity = 0
                for trait in asset["traits"]:
                    trait_count = int(trait["trait_count"])
                    if trait_count != 0:
                        rarity += 1 / (trait_count / items_in_collection)
                name = asset["name"]
                df.loc[idx] = [name, price, rarity, rarity / price]

In [ ]:
# Filter out top 20% in rarity
df[df["Rarity"] > df["Rarity"].min() + 0.1 * (df["Rarity"].max() - df["Rarity"].min())].sort_values("RarityPriceRatio", ascending=False)

In [ ]:
df.sort_values("RarityPriceRatio", ascending=False)